In [1]:
import sys; sys.path.insert(0, '..') # So that we import the local copy of pyzx if you have installed from Github
import os

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 

from multiprocessing import Pool
import pyzx as zx
from pyzx import CNOT_tracker # Note that this is the local pyzx package from this repository/fork, not the one installed through pip or other means.
from pyzx.routing import architecture
from pyzx import circuit
from pyzx import CombDecomposition
from pyzx.linalg import Mat2

Define a set of helper functions for importing circuits from qasm files, this code is based on some from pyzx_dynqubit/demos/PermRowCol

In [2]:
def read_circuit(source, parity_map=True):
    if not os.path.exists(source):
        print("File {} does not exist".format(source))
        return
    if parity_map:
        return CNOT_tracker.from_qasm_file(source)
    else:
        return circuit.Circuit.from_qasm_file(source)
def read_circuits(directory, n_qubits, parity_map=True):
    source_folder = f"../circuits/{directory}/"
    circuits = []
    sources = []
    subfolder = os.path.join(source_folder, str(n_qubits)+"qubits/")

    for folder in os.listdir(subfolder):
        folder = os.path.join(subfolder, folder)
        if os.path.isdir(folder):
            for file in os.listdir(folder):
                if file.endswith(".qasm"):
                    src = os.path.join(folder, file)
                    circuit = read_circuit(src, parity_map)
                    circuits.append(circuit)
                    sources.append(src)
    return circuits, sources

In [3]:
NUM_OF_QUBITS = 3
NUM_OF_GATES = 0
circuits = read_circuits("combs", NUM_OF_QUBITS, parity_map=False)
circuits = [circuits[0][x] for x in range(len(circuits[0])) if f"{NUM_OF_QUBITS}qubits/{NUM_OF_GATES}/" in circuits[1][x]]

In [4]:
old_circuit = circuits[0]
zx.draw(old_circuit)
comb_decomposition = CombDecomposition.from_circuit(old_circuit)

In [5]:
print(comb_decomposition.comb.matrix)
print(comb_decomposition.comb.holes)
zx.draw(comb_decomposition.comb)

[ 1  0  0  0  0 ]
[ 0  1  0  0  0 ]
[ 0  0  1  1  0 ]
[ 0  0  0  1  0 ]
[ 0  0  0  0  1 ]
bidict({1: 3, 0: 4})


In [6]:
def extract_sub_matrix(matrix, index_list):
    new_matrix = Mat2.zeros(len(index_list), len(index_list))
    for row in range(len(index_list)):
        for col in range(len(index_list)):
            new_matrix[col, row] = matrix[index_list[col], index_list[row]]
    return new_matrix
def insert_sub_matrix(matrix, sub_matrix, index_list):
    for row in range(len(index_list)):
        for col in range(len(index_list)):
            matrix[index_list[col], index_list[row]] = sub_matrix[col, row]
    return matrix

#print(comb_decomposition.comb.matrix)
#print(sub_matrix(comb_decomposition.comb.matrix, qubits_in_matrix))

In [7]:
matrix = comb_decomposition.comb.matrix.copy()
# Find initial qubits for the sub matrix
qubits_in_matrix = []
for qubit in range(comb_decomposition.comb.qubits):
    if qubit not in comb_decomposition.comb.holes.values():
        qubits_in_matrix.append(qubit)
print(qubits_in_matrix)
sub_matrix = extract_sub_matrix(matrix, qubits_in_matrix)
# Generate sub matrix
# Apply gaussian elimation procedure of our choosing to sub matrix
# Place sub matrix back in large matrix
insert_sub_matrix(matrix, sub_matrix, qubits_in_matrix)


no_more_holes = False
while not no_more_holes:
    # Update the qubits in the sub matrix to account for passing a hole
    for qubit_loc, qubit in enumerate(qubits_in_matrix):
        if qubit in comb_decomposition.comb.holes:
            qubits_in_matrix[qubit_loc] = comb_decomposition.comb.holes.pop(qubit)
    if len(comb_decomposition.comb.holes) == 0:
        no_more_holes = True
    print(qubits_in_matrix)
    sub_matrix = extract_sub_matrix(matrix, qubits_in_matrix)
    # Generate sub matrix
    # Apply gaussian elimation procedure of our choosing to sub matrix
    # Place sub matrix back in large matrix
    insert_sub_matrix(matrix, sub_matrix, qubits_in_matrix)
print(comb_decomposition.comb.matrix)
print("")
print(matrix)

[0, 1, 2]
[4, 3, 2]
[ 1  0  0  0  0 ]
[ 0  1  0  0  0 ]
[ 0  0  1  1  0 ]
[ 0  0  0  1  0 ]
[ 0  0  0  0  1 ]

[ 1  0  0  0  0 ]
[ 0  1  0  0  0 ]
[ 0  0  1  1  0 ]
[ 0  0  0  1  0 ]
[ 0  0  0  0  1 ]
